# Entropy pruning

## IMPORTS

In [1]:
import os
import tempfile

import numpy as np
import pandas as pd

from configparser import ConfigParser
os.environ["TF_USE_LEGACY_KERAS"]="1"

from tf_keras import losses,optimizers, models
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from utils import load_dataset, reformat_image, entropy_prune_model,time_benchmark, normalize_img, benchmark_tflite
from utils.Objects import EntropyPruningSurgeon, PrunedModel
from utils.Plots import plot_entropy_accuracy_threshold

---

## CONSTANTS

In [2]:
config = ConfigParser()
config.read("config.ini")

BATCH_SIZE = int(config["BATCHES"]["BATCH_SIZE"])

IMG_SIZE = (int(config["IMAGES"]["IMG_SIZE"]), int(config["IMAGES"]["IMG_SIZE"]))

MODEL_DIRECTORY = config["MODEL"]["MODEL_DIRECTORY"]
METRICS_DIRECTORY = config["MODEL"]["METRICS_DIRECTORY"]
WEIGHTS_DIRECTORY = config["MODEL"]["WEIGHTS_DIRECTORY"]

ENTROPY_METRICS = os.path.join(METRICS_DIRECTORY, config["MODEL"]["ENTROPY_METRICS"])
HORSE_AND_HUMAN_MODEL = os.path.join(MODEL_DIRECTORY, config["MODEL"]["HORSE_AND_HUMAN_MODEL"])
ENTROPY_HORSE_AND_HUMAN_WEIGHTS = os.path.join(WEIGHTS_DIRECTORY, config["MODEL"]["ENTROPY_HORSE_AND_HUMAN_WEIGHTS"])

## Import the dataset

In [4]:
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

INFO:absl:Load dataset info from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Constructing tf.data.Dataset horses_or_humans for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0


In [5]:
train_batches = train_examples.cache().shuffle(num_examples // 4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)

# Pruning the model

In [7]:
# Choose the best entropy level
k_entropies = np.arange(0.019, 0.031, 0.001)
metric_list = []

for k in k_entropies:
    # Load in the best saved model
    model_1 = models.load_model("model_experiments/custom_vgg16.keras")
    pruned_model: PrunedModel = entropy_prune_model(model_1, 20, k, train_batches, validation_batches, BATCH_SIZE)
    metrics = pruned_model.metrics
    val_loss = metrics["val_loss"]
    val_accuracy = metrics["val_accuracy"]

    print(f"Entropy={k}, \t {val_loss=}, \t {val_accuracy=}")
    metric_list.append(metrics)

# Create a dataframe of the values obtained
df = pd.DataFrame(metric_list)
df.to_csv(ENTROPY_METRICS)
df

Deleting 0/64 channels from layer: block1_conv1
Deleting 0/64 channels from layer: block1_conv2
Deleting 2/128 channels from layer: block2_conv1
Deleting 0/128 channels from layer: block2_conv2
Deleting 0/256 channels from layer: block3_conv1
Deleting 0/256 channels from layer: block3_conv2
Deleting 0/256 channels from layer: block3_conv3
Deleting 0/512 channels from layer: block4_conv1
Deleting 0/512 channels from layer: block4_conv2
Deleting 0/512 channels from layer: block4_conv3
Deleting 0/512 channels from layer: block5_conv1
Deleting 0/512 channels from layer: block5_conv2
Deleting 0/512 channels from layer: block5_conv3
Epoch 1/20



23/23 [==============================] - ETA: 0s - loss: 0.7517 - accuracy: 0.9736

KeyboardInterrupt: 

In [22]:
data = pd.read_csv(ENTROPY_METRICS)
plot_entropy_accuracy_threshold(data)

FileNotFoundError: [Errno 2] No such file or directory: 'metrics_experiments\\entropy_metrics.csv'

## Final Model Pruning Entropy

In [8]:
test_threshold = 0.028
final_model = models.load_model("model_experiments/custom_vgg16.keras")
entropy = EntropyPruningSurgeon(model=final_model, threshold=test_threshold)
pruned_model = entropy.run()
pruned_model.summary()

Deleting 0/64 channels from layer: block1_conv1
Deleting 0/64 channels from layer: block1_conv2
Deleting 3/128 channels from layer: block2_conv1
Deleting 0/128 channels from layer: block2_conv2
Deleting 0/256 channels from layer: block3_conv1
Deleting 0/256 channels from layer: block3_conv2
Deleting 0/256 channels from layer: block3_conv3
Deleting 374/512 channels from layer: block4_conv1
Deleting 265/512 channels from layer: block4_conv2
Deleting 278/512 channels from layer: block4_conv3
Deleting 280/512 channels from layer: block5_conv1
Deleting 252/512 channels from layer: block5_conv2
Deleting 255/512 channels from layer: block5_conv3
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 input_40 (InputLayer)       multiple                  0         
      

In [6]:
final_threshold = 0.026
final_model = models.load_model("model_experiments/custom_vgg16.keras")
pruned_model: PrunedModel = entropy_prune_model(final_model, 1, final_threshold, train_batches, validation_batches, BATCH_SIZE)
final_pruned_model = pruned_model.model
final_pruned_model.summary()

Deleting 0/64 channels from layer: block1_conv1
Deleting 0/64 channels from layer: block1_conv2
Deleting 3/128 channels from layer: block2_conv1
Deleting 0/128 channels from layer: block2_conv2
Deleting 0/256 channels from layer: block3_conv1
Deleting 0/256 channels from layer: block3_conv2
Deleting 0/256 channels from layer: block3_conv3
Deleting 237/512 channels from layer: block4_conv1
Deleting 98/512 channels from layer: block4_conv2
Deleting 99/512 channels from layer: block4_conv3
Deleting 39/512 channels from layer: block5_conv1
Deleting 48/512 channels from layer: block5_conv2
Deleting 109/512 channels from layer: block5_conv3



23/23 [==============================] - 155s 7s/step - loss: 4.4299 - accuracy: 0.8373 - val_loss: 0.2155 - val_accuracy: 0.9740
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 input_40 (InputLayer)       multiple                  0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 125)     72125 

In [7]:
# Test the model
final_pruned_model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
final_pruned_model.evaluate(validation_batches)
final_pruned_model.save_weights(ENTROPY_HORSE_AND_HUMAN_WEIGHTS)

 1/10 [==>...........................] - ETA: 40s - loss: 5.6955e-06 - accuracy: 1.0000

KeyboardInterrupt: 

In [8]:
val_data = validation_examples
val_data = val_data.map(reformat_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data = val_data.map(normalize_img)

# Benchmarking the model

In [16]:
original_model = models.load_model("model_experiments/custom_vgg16.keras")
init_time, avg_time, std = time_benchmark(final_pruned_model, class_names, val_data)

NameError: name 'val_data' is not defined

In [31]:
init_time, avg_time, std = time_benchmark(final_pruned_model, class_names, val_data)

1/1 [==============================] - 5s 5s/step
The first image takes 11302.82 ms
The average time taken per 99 images 7002.33 ms
The standard deviation of samples is 673.59 ms


# Transform the pruned model to TFLite

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_pruned_model)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmpefdfbgj2\assets


INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmpefdfbgj2\assets


Saved pruned TFLite model to: C:\Users\neyen\AppData\Local\Temp\tmpk3zqd946.tflite


# Test the TFLite model

In [13]:
import time
def benchmark_tflite(val_data, tflite_model, class_names, image_size):
    # Initialisation du modèle TensorFlow Lite
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    file_count = 0
    infer_times = []

    # Itération sur les données du Dataset
    for images, labels in val_data:
        # Vérification et traitement de chaque image du lot
        images = tf.image.resize(images, image_size)  # Redimensionner les images
        images = tf.cast(images, tf.float32)  # Convertir en float32
        images = images / 255.0  # Normalisation

        # Traiter chaque image dans le lot une par une
        for img in images:
            if file_count < 1:
                # Mesurer le temps d'initialisation pour la première image
                init_timer_start = time.time()
                img = np.expand_dims(img.numpy(), axis=0)  # Convertir en numpy et ajouter une dimension de lot
                interpreter.set_tensor(input_index, img)  # Passer à TensorFlow Lite
                interpreter.invoke()
                output = interpreter.tensor(output_index)
                pred_class = class_names[int(np.argmax(output()[0]))]
                init_timer_end = time.time()
                init_timer = init_timer_end - init_timer_start
                file_count += 1
            else:
                # Mesurer le temps d'inférence pour les images suivantes
                timer_start = time.time()
                img = np.expand_dims(img.numpy(), axis=0)  # Convertir en numpy et ajouter une dimension de lot
                interpreter.set_tensor(input_index, img)  # Passer à TensorFlow Lite
                interpreter.invoke()
                output = interpreter.tensor(output_index)
                pred_class = class_names[int(np.argmax(output()[0]))]
                timer_end = time.time()
                infer_times.append(timer_end - timer_start)
                file_count += 1

    # Calculer la moyenne et l'écart type des temps d'inférence
    avg_time = np.mean(infer_times) if infer_times else 0
    std_time = np.std(infer_times) if infer_times else 0

    return init_timer, avg_time, std_time

In [14]:
init_time, avg_time, std = benchmark_tflite(val_data,pruned_tflite_model,class_names=class_names,image_size=IMG_SIZE)
print(f"The first image takes {init_time * 1000:.2f} ms")
print(f"The average time taken per 99 images {avg_time * 1000:.2f} ms")
print(f"The standard deviation of samples is {std * 1000:.2f} ms")

The first image takes 588.21 ms
The average time taken per 99 images 572.16 ms
The standard deviation of samples is 157.33 ms


# Transform the quantized model to TFLite

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmpp88xadgv\assets


INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmpp88xadgv\assets


Saved quantized and pruned TFLite model to: C:\Users\neyen\AppData\Local\Temp\tmpvfi9_oyg.tflite


# Test the TFLite quantized model

In [17]:
init_time, avg_time, std = benchmark_tflite(val_data,quantized_and_pruned_tflite_model,class_names=class_names,image_size=IMG_SIZE)
print(f"The first image takes {init_time * 1000:.2f} ms")
print(f"The average time taken per 99 images {avg_time * 1000:.2f} ms")
print(f"The standard deviation of samples is {std * 1000:.2f} ms")

The first image takes 118.63 ms
The average time taken per 99 images 112.95 ms
The standard deviation of samples is 30.05 ms


# Test the size of the models

In [19]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [20]:
print("Size of gzipped baseline quantized pruned model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

# Calculate the difference in size
size_difference = get_gzipped_model_size(quantized_and_pruned_tflite_file) / get_gzipped_model_size(pruned_tflite_file)
print(f"The quantized and pruned model is {size_difference:.2f} times smaller than the pruned model.")

Size of gzipped baseline quantized pruned model: 8535115.00 bytes
Size of gzipped pruned Keras model: 38482073.00 bytes
The quantized and pruned model is 0.22 times smaller than the pruned model.
